In [3]:
# !pip install web3==5.17

In [11]:
import subprocess
import json
import os

from constants import BTC, BTCTEST, ETH
from pprint import pprint

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

# connect Web3
w3 = Web3(Web3.HTTPProvider(os.getenv('WEB3_PROVIDER', 'http://127.0.0.1:7545')))
# enable PoA middleware
# w3.middleware_stack.inject(geth_poa_middleware, layer=0)
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# set gas price strategy to built-in "medium" algorithm (est ~5min per tx)
# see https://web3py.readthedocs.io/en/stable/gas_price.html?highlight=gas
# see https://ethgasstation.info/ API for a more accurate strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

# including a mnemonic with prefunded test tokens for testing
mnemonic = os.getenv("MNEMONIC", "topple evolve spin party junk indicate exotic autumn federal donkey pact essence")
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --coin={coin} --numderive={depth} --format=json'
    print(command)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output)

def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account.address, "amount": value })
        return {
            "to": to,
            "from": account.address,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account.address),
            "chainId": w3.eth.chainId
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="toppl evolve spin party junk indicate exotic autumn federal donkey pact essence" --coin=eth --numderive=3 --format=json
php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="toppl evolve spin party junk indicate exotic autumn federal donkey pact essence" --coin=btc-test --numderive=3 --format=json


In [12]:
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0x3e47c6C5d0ab33f75423ef07476Ef14b2d0B0825',
   'xprv': 'xprvA3fzH1zGNFLuZMDfBMdVQN5ywPnKvgkvQweQNV18ugeimVVscE1YK48Bb55PPZ8vLthuELMLsFEjb7RHKWqp1cNhibg6FD3ZddjXJTYJMvG',
   'xpub': 'xpub6GfLgXXACcuCmqJ8HPAVmW2iVRcpL9UmnAa1AsQkU2BheHq29mKnrrSfSMsDnHGjNnWrF2XMaGzwmZ3HkZciyxEXLCB15THYmoTigFmpLke',
   'privkey': '0xfd9aa435d0aa51ae9ffd5c95ce88f51bfb1eedb80aa1606292c3b2c04aea1ac3',
   'pubkey': '03e55cb7811b20c55a3bc834204d947e3f3865eda19f020dc9093ce87b1415f718',
   'pubkeyhash': '7d7f9d9877b04cac665a8ba19e7f2043d6d65b82',
   'index': 0},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x14F977365DbE166A71B05FF87906B4a463869B6a',
   'xprv': 'xprvA3fzH1zGNFLub5L5yaysU6uaZr2CacqBHpmPBACc7rL4yEi2fwX8fgFBbFKCm7iTwRwyNqQnmVZiG7vqtyN8364ALr9PhsPFJ8napwWVH74',
   'xpub': 'xpub6GfLgXXACcuCoZQZ5cWsqErK7srgz5Z2f3gyyYcDgBs3r33BDUqPDUZfSXtZrF6u5s9adBbSJhbShivahW6JsGAKCQpL4eFPeDtbXNgjYf6',
   'privkey': '0xc7e3e7fdb1471a699721800136c3530dd7c9a88a3ec9de

In [13]:
for x in coins:
    print("-"*64)
    print(x)
    for y in coins[x]:
        print("privkey", y["privkey"])
        account = priv_key_to_account(x, y["privkey"])
        print(account.address)

----------------------------------------------------------------
eth
privkey 0xfd9aa435d0aa51ae9ffd5c95ce88f51bfb1eedb80aa1606292c3b2c04aea1ac3
0x3e47c6C5d0ab33f75423ef07476Ef14b2d0B0825
privkey 0xc7e3e7fdb1471a699721800136c3530dd7c9a88a3ec9deb5c2c11da1855d0633
0x14F977365DbE166A71B05FF87906B4a463869B6a
privkey 0x5802feeea8b513a187b653069ce0439b68d1a6886cd562d80fdf1718aab2d99c
0x895BC7f1336C6c19C8d7577f7e14CdB3Bdf692C2
----------------------------------------------------------------
btc-test
privkey cNwnMuUon2Ak4QcPxTR1JNQQMV6x8dLDMiSNWPqJbyMBaWc8F2vE
mrFiFfgtgtRuwtohKxAdwF7HEPVqjQwnnq
privkey cQ6HmaAQa9JDiuhV2G8RWEFy5XZ3qPK5C2qrFy4KtfXXUK2Bgn5R
mhrcgVgzUsh8psmrnFsHabdov74BAV7s86
privkey cW3pLm2AETENpo2kyEF1osaDu2vaZ2zjY3qKk6jcHYGggUEwfwtY
muuD9tFS1wjEcJ4ah2ehw7kzxAdumeXJBS


In [14]:
account = priv_key_to_account("eth", coins["eth"][0]["privkey"])
account

In [15]:
send_tx(ETH, account, '0x14F977365DbE166A71B05FF87906B4a463869B6a', 1.1)

HexBytes('0x8aeaa89d3334be56ccb37a87f2670e8bb02e659caffeb19b2c8af604dd4195a0')

In [28]:
account = priv_key_to_account("btc-test", coins["btc-test"][0]["privkey"])
account

<PrivateKeyTestnet: mrFiFfgtgtRuwtohKxAdwF7HEPVqjQwnnq>

In [33]:
send_tx("btc-test", account, 'mhrcgVgzUsh8psmrnFsHabdov74BAV7s86', 0.0000000000000000000000001)

InsufficientFunds: Balance 5682 is less than 23052 (including fee).